In [ ]:
import gym
import numpy as np
import cma
import math
import random
import matplotlib.pyplot as plt
import numpy
import operator
from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp
from numpy import linalg as LA
#Trés optionnel
import networkx as nx
from IPython.display import Image, display


# Robotique : Gym
Dans cette partie nous allons utiliser gym, une librairie permettant de manipuler des environnements physiques facilement en vue d'expérimenter des algorithmes permettant d'apprendre efficacement des politiques.

In [ ]:
env = gym.make('CartPole-v1')
observation = env.reset()
total_reward = 0
for t in range(100):
    action = np.random.rand(1,)
    if action > 0:
        action = 1
    else:
        action = 0
    observation, reward, done, info = env.step(action)
    print(observation)
    total_reward += reward
    if done:
        print("Episode finished after %d timesteps" % (t + 1))
        break

On voit bien qu'avec une politique aléatoire le cart tiens seulement 9 timesteps a l'écran.
Dans ce qui suit nous allons entrainer un controller afin de controller le carte et de le faire rester le plus longtemps possible a l'écran.
La structure du controller serra évidente : 
- Etant donné que l'observation se compose de 4 données (position, velocity, angle, velocity du cable) le réseau de neurones prendra ces 4 paramètres en entrée
- La sortie serra tanh et ferra dépendre l'action a prendre
- La fonction d'évaluation serra simplement le reward total cumulé.

In [ ]:
def sigmoid(x):
    return 1./(1 + np.exp(-x))

def tanh(x):
    return np.tanh(x)


def gen_simplemlp(n_in, n_out, n_hidden_layers=2, n_neurons_per_hidden=5):
    n_neurons = [n_neurons_per_hidden]*n_hidden_layers if np.isscalar(n_neurons_per_hidden) else n_neurons_per_hidden
    i = Input(shape=(n_in,))
    x = i
    for n in n_neurons:
        x = Dense(n, activation='sigmoid')(x)
    o = Dense(n_out, activation='tanh')(x)
    m = Model(inputs=i, outputs=o)
    return m
    

class SimpleNeuralControllerNumpy():
    def __init__(self, n_in, n_out, n_hidden_layers=2, n_neurons_per_hidden=5, params=None):
        self.dim_in = n_in
        self.dim_out = n_out
        # if params is provided, we look for the number of hidden layers and neuron per layer into that parameter (a dicttionary)
        if (not params==None):
            if ("n_hidden_layers" in params.keys()):
                n_hidden_layers=params["n_hidden_layers"]
            if ("n_neurons_per_hidden" in params.keys()):
                n_neurons_per_hidden=params["n_neurons_per_hidden"]
        self.n_per_hidden = n_neurons_per_hidden
        self.n_hidden_layers = n_hidden_layers
        self.weights = None 
        self.n_weights = None
        self.init_random_params()
        self.out = np.zeros(n_out)
        #print("Creating a simple mlp with %d inputs, %d outputs, %d hidden layers and %d neurons per layer"%(n_in, n_out,n_hidden_layers, n_neurons_per_hidden))

    
    def init_random_params(self):
        if(self.n_hidden_layers > 0):
            self.weights = [np.random.random((self.dim_in,self.n_per_hidden))] # In -> first hidden
            self.bias = [np.random.random(self.n_per_hidden)] # In -> first hidden
            for i in range(self.n_hidden_layers-1): # Hidden -> hidden
                self.weights.append(np.random.random((self.n_per_hidden,self.n_per_hidden)))
                self.bias.append(np.random.random(self.n_per_hidden))
            self.weights.append(np.random.random((self.n_per_hidden,self.dim_out))) # -> last hidden -> out
            self.bias.append(np.random.random(self.dim_out))
        else:
            self.weights = [np.random.random((self.dim_in,self.dim_out))] # Single-layer perceptron
            self.bias = [np.random.random(self.dim_out)]
        self.n_weights = np.sum([np.product(w.shape) for w in self.weights]) + np.sum([np.product(b.shape) for b in self.bias])

    def get_parameters(self):
        """
        Returns all network parameters as a single array
        """
        flat_weights = np.hstack([arr.flatten() for arr in (self.weights+self.bias)])
        return flat_weights

    def set_parameters(self, flat_parameters):
        """
        Set all network parameters from a single array
        """
        i = 0 # index
        to_set = []
        self.weights = list()
        self.bias = list()
        if(self.n_hidden_layers > 0):
            # In -> first hidden
            w0 = np.array(flat_parameters[i:(i+self.dim_in*self.n_per_hidden)])
            self.weights.append(w0.reshape(self.dim_in,self.n_per_hidden))
            i += self.dim_in*self.n_per_hidden
            for l in range(self.n_hidden_layers-1): # Hidden -> hidden
                w = np.array(flat_parameters[i:(i+self.n_per_hidden*self.n_per_hidden)])
                self.weights.append(w.reshape((self.n_per_hidden,self.n_per_hidden)))
                i += self.n_per_hidden*self.n_per_hidden
            # -> last hidden -> out
            wN = np.array(flat_parameters[i:(i+self.n_per_hidden*self.dim_out)])
            self.weights.append(wN.reshape((self.n_per_hidden,self.dim_out)))
            i += self.n_per_hidden*self.dim_out
            # Samefor bias now
            # In -> first hidden
            b0 = np.array(flat_parameters[i:(i+self.n_per_hidden)])
            self.bias.append(b0)
            i += self.n_per_hidden
            for l in range(self.n_hidden_layers-1): # Hidden -> hidden
                b = np.array(flat_parameters[i:(i+self.n_per_hidden)])
                self.bias.append(b)
                i += self.n_per_hidden
            # -> last hidden -> out
            bN = np.array(flat_parameters[i:(i+self.dim_out)])
            self.bias.append(bN)
            i += self.dim_out
        else:
            n_w = self.dim_in*self.dim_out
            w = np.array(flat_parameters[:n_w])
            self.weights = [w.reshape((self.dim_in,self.dim_out))]
            self.bias = [np.array(flat_parameters[n_w:])]
        self.n_weights = np.sum([np.product(w.shape) for w in self.weights]) + np.sum([np.product(b.shape) for b in self.bias])
    
    def predict(self,x):
        """
        Propagage
        """
        if(self.n_hidden_layers > 0):
            #Input
            a = np.matmul(x,self.weights[0]) + self.bias[0]
            y = sigmoid(a)
            # hidden -> hidden
            for i in range(1,self.n_hidden_layers-1):
                a = np.matmul(y, self.weights[i]) + self.bias[i]
                y = sigmoid(a)
            # Out
            a = np.matmul(y, self.weights[-1]) + self.bias[-1]
            out = tanh(a)
            return out
        else: # Simple monolayer perceptron
            return tanh(np.matmul(x,self.weights[0]) + self.bias[0])

    def __call__(self,x):
        """Calling the controller calls predict"""
        return self.predict(x)


nn = SimpleNeuralControllerNumpy(4, 1, 2, 5)


In [ ]:
def eval_nn(genotype, render=True):
    total_reward=0
    nn=SimpleNeuralControllerNumpy(4,1,2,5)
    nn.set_parameters(genotype)
    observation = env.reset()
    for t in range(1000):
        if render:
            env.render()
        action=nn.predict(observation)
        if action>0:
            action=1
        else:
            action=0
        observation, reward, done, info = env.step(action)
        total_reward+=reward
        if done:
            print("Episode finished after %d timesteps"%(t+1))
            break
    return -total_reward

In [ ]:
import cma.purecma as purecma
nn = SimpleNeuralControllerNumpy(4, 1, 2, 5)
nn.init_random_params()
w = nn.get_parameters()
res = cma.CMAEvolutionStrategy(w, 0.5).optimize(eval_nn,maxfun=10000).result


# Programmation Génétique :

In [ ]:
import math
import random

import numpy
import operator
from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

In [ ]:
def protectedDiv(left, right):
    try:
        return left / right
    except ZeroDivisionError:
        return 1

pset = gp.PrimitiveSet("MAIN", 1)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(protectedDiv, 2)
pset.addPrimitive(operator.neg, 1)
pset.addPrimitive(math.cos, 1)
pset.addPrimitive(math.sin, 1)
pset.addEphemeralConstant("rand101", lambda: random.randint(-1,1))
pset.renameArguments(ARG0='x')

In [ ]:



creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)


def f(x):
    d = x**4 - x**3 + x**2 + np.sin(2*x+3)
    return d

def evalSymbReg(individual, points):
    # Transform the tree expression in a callable function
    func = toolbox.compile(expr=individual)
    # Evaluate the mean squared error between the expression
    # and the real function : x**4 + x**3 + x**2 + x
    sqerrors = ((func(x) - f(x))**2 for x in points)
    return math.fsum(sqerrors) / len(points),



toolbox.register("evaluate", evalSymbReg, points=[x/10. for x in range(-10,10)])
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))




pop = toolbox.population(n=300)
hof = tools.HallOfFame(1)
    
stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
stats_size = tools.Statistics(len)
mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
mstats.register("avg", numpy.mean)
mstats.register("std", numpy.std)
mstats.register("min", numpy.min)
mstats.register("max", numpy.max)
pop, log = algorithms.eaSimple(pop, toolbox, 0.5, 0.1, 172, stats=mstats,halloffame=hof, verbose=True)


In [ ]:
nodes, edges, labels = gp.graph(hof[0])

print(nodes)
print(edges)
print(labels)
sommets = []
arcs = [(str(labels[i])+str(i),str(labels[j])+str(j)) for i,j in edges]
for i,j in arcs:
    if not i in sommets:
        sommets.append(i)
    if not j in sommets:
        sommets.append(j)

G = nx.MultiDiGraph()
G.add_nodes_from(sommets )
G.add_edges_from(arcs)

#for i in nodes:
#    n = G.get_node(i)
#    n.attr["label"] = labels[i]
#

from IPython.display import Image, display

def view_pydot(pdot):
    plt = Image(pdot.create_png())
    display(plt)

import networkx as nx
to_pdot = nx.drawing.nx_pydot.to_pydot
pdot = to_pdot(G)
view_pydot(pdot)

In [ ]:
import gym
import numpy as np
import cma
import math
import random
import matplotlib.pyplot as plt
import numpy
import operator
from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp
from numpy import linalg as LA
#Trés optionnel
import networkx as nx
from IPython.display import Image, display


x = np.linspace(-3,3,100).reshape((-1,1))
y = np.sin(x) + np.random.normal(0,0.1,x.shape)

def adapt(X):
    return np.hstack((X,np.ones((X.shape[0],1))))  

def predictions(w,x):
    ypredict = np.dot(x,w.T)
    return ypredict

w = np.dot(np.linalg.pinv(adapt(x)),y)
print("Donc le modèle qu'on trouve est : ",w.T)

plt.scatter(x,y)
#Calcul des prédictions
ypred = predictions(w.T,adapt(x))
#Tracage du modèle : 
plt.plot(x,ypred)
plt.show()

In [ ]:
def radial_basis_func(X,center=0.5,widths=1):
    c = X[int(center*X.shape[0])]
    return np.exp(-np.divide(np.square(x - c), widths))

recepient = [(0.2,0.5),(0.5,1),(0.8,1)]

def kernel(X,recipients):
    H = radial_basis_func(X,center = recipients[0][0],widths=recipients[0][1])
    for r in recipients[1:]:
        H = np.hstack((H,radial_basis_func(X,center = r[0],widths=r[1])))
    return H

def display_recipients(X,reciepients):
    for r in reciepients:
        y = radial_basis_func(X,center = r[0],widths=r[1])
        plt.plot(X,y)
    plt.show()

b1 = radial_basis_func(x,center = 0.2)
plt.plot(x,b1)
plt.show()
print("b1 = ",b1.shape)
print("x = ",x.shape)


w = np.dot(np.linalg.pinv(kernel(x,recepient).T.dot(kernel(x,recepient))),kernel(x,recepient).T.dot(y))


print("Donc le modèle qu'on trouve est : ",w.T)
plt.scatter(x,y)
#Calcul des prédictions
ypred = predictions(w.T,kernel(x,recepient))
#Tracage du modèle : 
plt.plot(x,ypred)
plt.show()

In [ ]:
x = np.linspace(-3,3,100).reshape((-1,1))
y = np.sin(x) + np.random.normal(0,0.1,x.shape)

recepient = [(0.2,1),(0.5,4),(0.8,1)]
display_recipients(x,recepient)

w = np.dot(np.linalg.pinv(kernel(x,recepient).T.dot(kernel(x,recepient))),kernel(x,recepient).T.dot(y))


print("Donc le modèle qu'on trouve est : ",w.T)
plt.scatter(x,y)
#Calcul des prédictions
ypred = predictions(w.T,kernel(x,recepient))
#Tracage du modèle : 
plt.plot(x,ypred)
plt.show()

In [ ]:
def kernel(X,recipients):
    H = radial_basis_func(X,center = recipients[0][0],widths=recipients[0][1])
    for r in recipients[1:]:
        H = np.hstack((H,radial_basis_func(X,center = r[0],widths=r[1])))
    return H

def adding_kernel(X,rec):
    H = np.hstack((X,radial_basis_func(X,center = rec[0],widths=rec[1])))
    return H

def identity(x):
    return x

def recipien_generator():
    return tuple([np.random.rand(1)[0],np.random.uniform(1,10)])

pset2 = gp.PrimitiveSetTyped("KERN_MAIN", [np.ndarray],np.ndarray)
pset2.addPrimitive(adding_kernel, [np.ndarray, tuple], np.ndarray)
pset2.addPrimitive(identity, [tuple], tuple)

pset2.addEphemeralConstant("recipient2",recipien_generator , tuple)

pset2.renameArguments(ARG0='x')

In [ ]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset2, min_=1, max_=3)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset2)

In [ ]:
from sklearn.metrics import mean_squared_error

X = np.linspace(-3,3,100).reshape((-1,1))
y = np.sin(x) + np.random.normal(0,0.1,x.shape)

def eval_kernel(individual):
    # Transform the tree expression in a callable function
    func = toolbox.compile(expr=individual)
    K = func(X)
    try:
        w = np.dot(np.linalg.pinv(K.T.dot(K)),K.T.dot(y))
        ypred = predictions(w.T,K)
        fitness = mean_squared_error(y,ypred)
    except(np.linalg.LinAlgError):
        fitness = 1e20
    return fitness,



toolbox.register("evaluate", eval_kernel)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset2)
toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))




In [ ]:
pop = toolbox.population(n=2)
hof = tools.HallOfFame(1)
    
stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
stats_size = tools.Statistics(len)
mstats = tools.MultiStatistics(fitness=stats_fit)
mstats.register("avg", numpy.mean)
mstats.register("std", numpy.std)
mstats.register("min", numpy.min)
mstats.register("max", numpy.max)

pop,log= algorithms.eaMuPlusLambda(pop, toolbox, mu=20, lambda_=100, cxpb=0, mutpb=0.3, ngen=200, stats=mstats,halloffame=hof, verbose=True)


In [ ]:
nodes, edges, labels = gp.graph(hof[0])

print(nodes)
print(edges)
print(labels)
sommets = []
arcs = [(str(labels[i])+str(i),str(labels[j])+str(j)) for i,j in edges]
for i,j in arcs:
    if not i in sommets:
        sommets.append(i)
    if not j in sommets:
        sommets.append(j)

G = nx.MultiDiGraph()
G.add_nodes_from(sommets )
G.add_edges_from(arcs)

#for i in nodes:
#    n = G.get_node(i)
#    n.attr["label"] = labels[i]
#

from IPython.display import Image, display

def view_pydot(pdot):
    plt = Image(pdot.create_png())
    display(plt)

import networkx as nx
to_pdot = nx.drawing.nx_pydot.to_pydot
pdot = to_pdot(G)
view_pydot(pdot)

In [ ]:
func = toolbox.compile(expr=hof[0])
K = func(X)
w = np.dot(np.linalg.pinv(K.T.dot(K)),K.T.dot(y))
#Calcul des prédictions
ypred = predictions(w.T,K)
plt.scatter(X,y)
#Tracage du modèle : 
plt.plot(X,ypred)
plt.show()

# Résolution d'un problème SAT 

In [ ]:
from pysat.formula import CNF
import numpy as np


formula = CNF(from_file='input1.cnf')
clause = formula.clauses
L = []
for c in clause:
    L.append(np.array(c))
L = np.array(L)
print(L)

def transform(clause):
    L = []
    for c in clause:
        L.append(np.array(c))
    L = np.array(L)
    return L



def nb_sat(valuation):
    n = 0
    valuation = np.array(valuation)
    for clause in L:
        if(len(clause)!=0):
            a = valuation[clause]
            for c,cl in zip(a,clause):
                if(c == 1 and cl>0):
                    n = n + 1
                    break
                if(c == 0 and cl<0):
                    n = n + 1
                    break
    return n,

def count_clauses():
    n = 0
    for clause in L:
        if(len(clause)!=0):
            n = n + 1
    return n


nb_sat(np.array((formula.nv+1)*[1]))
print(count_clauses())

In [ ]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
# Attribute generator 
toolbox.register("attr_bool", random.randint, 0, 1)
# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_bool, formula.nv+1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", nb_sat)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=6)

pop = toolbox.population(n=100)
stats = tools.Statistics(key=lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)
l=algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.3, ngen=1000,stats=stats)
#l= algorithms.eaMuPlusLambda(pop, toolbox, mu=50, lambda_=100, cxpb=0.4, mutpb=0.3, ngen=1000, stats=stats)

# Utilisation d'une évaluation multi-critère


## Introduction a la décision multi-critères 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
datas = [
    dict(
        x = 10,
        y = 12
    ),
    dict(
        x = 15,
        y = 11
    ),
    dict(
        x = 8,
        y = 8
    ),
    dict(
        x = 17,
        y = 5
    ),
    dict(
        x = 6,
        y = 18
    ),
    dict(
        x = 8,
        y = 12
    ),
    dict(
        x = 12,
        y = 16
    ),
    dict(
        x = 10,
        y = 17
    ),
    dict(
        x = 12,
        y = 10
    ),
]
X = [d["x"] for d in datas]
Y = [d["y"] for d in datas]
df = pd.DataFrame(datas)
print(df)
plt.scatter(X,Y)

In [ ]:
def calculer_front(X):
    L = X
    for i in X:
        for j in L:
            if(i["x"] >= j["x"] and i["y"] > j["y"] or i["x"] > j["x"] and i["y"] >= j["y"]):
                L.remove(j)
            if(j["x"] >= i["x"] and j["y"] > i["y"] or j["x"] > i["x"] and j["y"] >= i["y"]):
                L.remove(i)
    return L


In [ ]:
X = [d["x"] for d in datas]
Y = [d["y"] for d in datas]
plt.scatter(X,Y)
calculer_front(datas)
X = [d["x"] for d in datas]
Y = [d["y"] for d in datas]
plt.scatter(X,Y,color = "red")
plt.show()


## Algorithmes evolutionnaires Multi-critères NSGA2 et NSGA3

In [ ]:
nn = SimpleNeuralControllerNumpy(4, 1, 2, 5)
nn.init_random_params()
w = nn.get_parameters()

creator.create("MaFitness", base.Fitness, weights=(-1.0,-1.0,+2.0))
creator.create("Individual", list, fitness=creator.MaFitness)

toolbox = base.Toolbox()

# à compléter pour sélectionner les opérateurs de mutation, croisement, sélection avec des toolbox.register(...)

# Attribute generator
toolbox.register("attr_float", random.uniform, -2, 2)

# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, len(w))
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", tools.cxSimulatedBinary, eta = 0.5)
toolbox.register("mutate", tools.mutPolynomialBounded, indpb=0.05, eta = 15, low = -5, up = 5)
toolbox.register("select", tools.selTournament, tournsize=3)

# Les statistiques permettant de récupérer les résultats
stats_fit = tools.Statistics(lambda ind: ind.fitness.values[2])
stats_size = tools.Statistics(lambda ind: -ind.fitness.values[0])
mstats = tools.MultiStatistics(temps=stats_fit, erreur=stats_size)
mstats.register("avg", numpy.mean)
mstats.register("max", numpy.max)
# La structure qui permet de stocker les statistiques
logbook = tools.Logbook()

# La structure permettant de récupérer le meilleur individu
hof = tools.HallOfFame(1)

## à compléter pour initialiser l'algorithme, n'oubliez pas de mettre à jour les statistiques, le logbook et le hall-of-fame.

pop = toolbox.population(n=100)
CXPB, MUTPB = 0.5, 0.2

hof.update(pop)

In [ ]:
env = gym.make('CartPole-v1')
env._max_episode_steps = 500
def eval_nn(genotype, render=True, nbstep=500, doPrint = True):
    global errors
    global position
    observation = env.reset()
    total_reward = 0
    total_x=0 # l'erreur en x est dans observation[0]
    total_theta=0 #  l'erreur en theta est dans obervation[2]
    nn=SimpleNeuralControllerNumpy(4,1,2,5)
    nn.set_parameters(genotype)
    maxXerr = 0
    maxThetaerr = 0
    ti = 0
    observation = env.reset()
    for t in range(nbstep):
        ti = t
        if render:
            env.render()
        action = nn.predict(observation)
        if action > 0:
            action = 1
        else:
            action = 0
        observation, reward, done, info = env.step(action)
        total_x += np.abs(observation[0]-0.2)
        total_theta += np.abs(observation[2])
        if(done):
            break
        if maxXerr < np.abs(observation[0]) :
            maxXerr = np.abs(observation[0])
        if maxThetaerr < np.abs(observation[2]) :
            maxThetaerr = np.abs(observation[2])
    return (total_x/t, total_theta/t, t)

toolbox.register("evaluate", eval_nn)
l=algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.3, ngen=1000,stats=mstats,verbose=True,)

